In [2]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/2AGxwFF-qCwfZF1SwmJJgD2Qyvum7Kf9Envwm39U8YERKtacfSQ1E_g

Successfully saved authorization token.


In [27]:
# Import the Image function from the IPython.display module. 
from IPython.display import Image

### Simple way to get image

In [4]:
image = ee.Image('srtm90_v4')
print(image.getInfo())
Image(url = image.getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs': 'EPSG:4326', 'crs_transform': [0.000833333333333, 0, -180, 0, -0.000833333333333, 60]}], 'version': 1494271934303000.0, 'id': 'srtm90_v4', 'properties': {'system:time_start': 950227200000, 'system:time_end': 951177600000, 'system:asset_size': 18827626666}}


In [5]:
# Get a download URL for an image.
image1 = ee.Image('srtm90_v4')
path = image1.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]'
})
print(path)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6372828bf91c925d155cb0ea8c792d61-0fb3e16e858e16dea80575c442f39614:getPixels


## Using thirdparty packages

In [6]:
import folium
import geehydro
from datetime import datetime as dt

In [7]:
# the Ituna/Itatá Indigenous Land in Brazil.
Ituna_map = folium.Map(location=[-4.06738, -52.034], zoom_start=10)
# Ituna_map

In [8]:
# get data from landsat-8
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# setting the Area of Interest (AOI)
Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
                                   -52.23999, -4.38201])
# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)


# choose dates
# landsat = landsat.filterDate('2019-07-01','2019-12-01')
# landsat_AOI.getInfo()

# choose image
# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())
# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

Cloud Cover (%): 0


In [9]:
# visualizing satellite imagery
# setting the Area of Interest (AOI)
# Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
#                                    -52.23999, -4.38201])
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}
             
Image(url = least_cloudy.getThumbUrl(parameters))

## Image collections

**Landsat-8**

In [10]:
# get data from landsat-8
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# setting the Area of Interest (AOI)
sat_AOI = ee.Geometry.Rectangle([137.3, 34.65, 137.5, 34.85])

# filter area
landsat8_AOI = landsat8.filterBounds(sat_AOI)

least_cloudy8 = ee.Image(landsat8_AOI.sort('CLOUD_COVER').first())

parameters8 = {'min': 0,
              'max': 3000,
              'dimensions': 1024*3,
              'bands': ['B4', 'B3', 'B2'],
              'region': sat_AOI}
             
Image(url = least_cloudy8.getThumbUrl(parameters8))

**Sentinel-2 MSI: MultiSpectral Instrument, Level-2A**

In [11]:
def maskS2clouds(image):
    """
     Function to mask clouds using the Sentinel-2 QA band
     @param {ee.Image} image Sentinel-2 image
     @return {ee.Image} cloud masked Sentinel-2 image
     """
    qa = image.select('QA60');

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = (1 << 10)
    cirrusBitMask = (1 << 11)

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

In [30]:
landsat = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterDate('2018-01-01', '2018-12-31')\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
            .map(maskS2clouds)
# setting the Area of Interest (AOI)
# all region
# Ituna_AOI = ee.Geometry.Rectangle([137.3, 34.65, 137.5, 34.85])
# very small region  34.7,34.75,137.325,137.35
# sat_region = ee.Geometry.Rectangle([137.3, 34.65, 137.325, 34.675])
# sat_region = ee.Geometry.Rectangle([137.325, 34.66, 137.375, 34.70])
sat_region = ee.Geometry.Rectangle([137.3, 34.66, 137.375, 34.70])
34.65, 137.3, 34.675, 137.32500000000002

# filter area
landsat_AOI = landsat.filterBounds(sat_region)

least_cloudy = ee.Image(landsat_AOI.sort('LAND_COVER').first())

parameters = {'min': 0,
              'max': 0.3,
              # 'dimensions': 1024*3,
              'scale':10,
              'bands': ['B4', 'B3', 'B2'],
              'region': sat_region}
             
Image(url = least_cloudy.getThumbUrl(parameters))

In [ ]:
# export image
color_image = landsat_AOI.select(['B4', 'B3', 'B2'])
color_image = ee.Image(landsat_AOI.sort('LAND_COVER').first())
task_config = {
    # 'description': 'imageToDriveExample',
    'scale': 5,
    'region': Ituna_AOI, #["coordinates"][0]
    'folder':'map_exports',
    }
task = ee.batch.Export.image(color_image, 'toyohashi_export_image', task_config)
task.start()

In [13]:
# another export format
landsat = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterDate('2018-01-01', '2018-12-31')\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))\
            .map(maskS2clouds).select(['B4', 'B3', 'B2'])
# setting the Area of Interest (AOI)
# Ituna_AOI = ee.Geometry.Rectangle([137.3, 34.65, 137.5, 34.85])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.35, 34.7])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.35, 34.675])
# Ituna_AOI = ee.Geometry.Rectangle([137.325, 34.65, 137.33, 34.655])
# Ituna_AOI = ee.Geometry.Rectangle([137.3, 34.65, 137.325, 34.675])
Ituna_AOI = ee.Geometry.Rectangle([137.3125, 34.6625, 137.325, 34.675])
# 137.3:137.325, 34.65:34.675
# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)

least_cloudy = ee.Image(landsat_AOI.sort('LAND_COVER').first())
task_config = {
    'scale': 1,
    'region': Ituna_AOI,
    'folder':'map_exports',
    'crs': 'EPSG:4326'
    }
task = ee.batch.Export.image(least_cloudy, 'toyohashi_export_full_scale_smaller_normed', task_config)
task.start()

#### export tiles of the toyohashi area
Coordinates:
34.65 - 34.85
137.3 - 137.5

Resolution:
0.0125 * 0.0125

In [14]:
import numpy as np

In [121]:
def grid_pairs_old(min_x, max_x, min_y, max_y, resolution):
    # transform to origin
    max_x -= min_x
    max_y -= min_y
    grid_x, grid_y = np.mgrid[0:max_x+resolution:resolution, 0:max_y+resolution:resolution]
    
    grid_points = list(zip(grid_x.flatten(), grid_y.flatten()))
#     print(grid_points)
    skips = int(max_x/resolution)+2
#     print("skips: ", skips)
    count = 0
    pairs_coords = []
    for i, point in enumerate(grid_points[:-skips]):
        if point[1]==max_x:
            continue
        count+=1
#         print(point)
        pairs_coords.append([point[0]+min_x, point[1]+min_y, grid_points[i+skips][0]+min_x, grid_points[i+skips][1]+min_y])
    return pairs_coords

def grid_pairs(min_x, max_x, min_y, max_y, resolution):
    # transform to origin
    nx, ny = (int((max_x-min_x)/resolution)+1, int((max_y-min_y)/resolution)+1)
    x = np.linspace(min_x, max_x, nx)
    y = np.linspace(min_y, max_y, ny)
    xv, yv = np.meshgrid(x, y)
    grid_points = list(zip(yv.flatten(), xv.flatten()))
    # print(grid_points)
    skips = int((max_x-min_x)/resolution)+2
    count = 0
    pairs_coords = []
    for i, point in enumerate(grid_points[:-skips]):
        if point[1]==max_x:
            continue
        count+=1
        pairs_coords.append([point[0], point[1], grid_points[i+skips][0], grid_points[i+skips][1]])
    return pairs_coords
# grid_pairs(0,0.1,0,0.1,0.05)
# grid_pairs(0,1,0,1,0.5)
# grid_pairs(0,1,0,1,0.5)
grid_pairs(34.65,34.7,137.35,137.4,0.0125*2)

[[137.35, 34.65, 137.375, 34.675],
 [137.35, 34.675, 137.375, 34.7],
 [137.375, 34.65, 137.4, 34.675],
 [137.375, 34.675, 137.4, 34.7]]

In [114]:
grid_pairs(0,0.1,0,0.1,0.05)

[(0.0, 0.0), (0.0, 0.05), (0.0, 0.1), (0.05, 0.0), (0.05, 0.05), (0.05, 0.1), (0.1, 0.0), (0.1, 0.05), (0.1, 0.1)]


[[0.0, 0.0, 0.05, 0.05],
 [0.0, 0.05, 0.05, 0.1],
 [0.05, 0.0, 0.1, 0.05],
 [0.05, 0.05, 0.1, 0.1]]

In [115]:
# coords = grid_pairs(34.65,34.85,137.3,137.5,0.0125*2)
# coords = grid_pairs(34.7,34.75,137.325,137.35,0.0125*2) # for greenhouse sample images
# coords = grid_pairs(34.66,34.70,137.325,137.375,0.0125*2) # batch 2 greenhouse sample images
print(len(coords))

12


In [17]:
from time import sleep

In [132]:
# grid_pairs(34.7,34.75,137.35,137.375,0.0125*2)
# grid_pairs(0,1,0,2,1)
len(grid_pairs(34.65,34.85,137.3,137.5,0.025))

56

In [128]:
# coords = grid_pairs(34.65,34.85,137.3,137.5,0.025) # all tiles
coords = grid_pairs(34.675,34.725,137.35,137.4,0.025) # greenhouse samples

# another export format
landsat = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterDate('2018-01-01', '2018-12-31')\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
            .map(maskS2clouds).select(['B4', 'B3', 'B2'])

# export Area of Interests (AOI)
count = 0
for coord in coords:
    name = "{:.4f}_{:.4f}".format(coord[0], coord[1])
    count += 1
    print(f"{count}: {name}")
    export_AOI = ee.Geometry.Rectangle(coord)

    # filter area
    landsat_AOI = landsat.filterBounds(export_AOI)

    least_cloudy = ee.Image(landsat_AOI.sort('LAND_COVER').first())
    
    task_config = {
        'scale': 1,
        'region': export_AOI,
        'folder':'toyohashi_all_tiles',
        'crs': 'EPSG:4326'
        }
    task = ee.batch.Export.image(least_cloudy, name, task_config)
    task.start()
    sleep(1)

1: 137.3000_34.6500
2: 137.3000_34.6750
3: 137.3000_34.7000
4: 137.3000_34.7250
5: 137.3000_34.7500
6: 137.3000_34.7750
7: 137.3000_34.8000
8: 137.3000_34.8250
9: 137.3286_34.6500
10: 137.3286_34.6750
11: 137.3286_34.7000
12: 137.3286_34.7250
13: 137.3286_34.7500
14: 137.3286_34.7750
15: 137.3286_34.8000
16: 137.3286_34.8250
17: 137.3571_34.6500
18: 137.3571_34.6750
19: 137.3571_34.7000
20: 137.3571_34.7250
21: 137.3571_34.7500
22: 137.3571_34.7750
23: 137.3571_34.8000
24: 137.3571_34.8250
25: 137.3857_34.6500
26: 137.3857_34.6750
27: 137.3857_34.7000
28: 137.3857_34.7250
29: 137.3857_34.7500
30: 137.3857_34.7750
31: 137.3857_34.8000
32: 137.3857_34.8250
33: 137.4143_34.6500
34: 137.4143_34.6750
35: 137.4143_34.7000
36: 137.4143_34.7250
37: 137.4143_34.7500
38: 137.4143_34.7750
39: 137.4143_34.8000
40: 137.4143_34.8250
41: 137.4429_34.6500
42: 137.4429_34.6750
43: 137.4429_34.7000
44: 137.4429_34.7250
45: 137.4429_34.7500
46: 137.4429_34.7750
47: 137.4429_34.8000
48: 137.4429_34.8250
4

### NAIP: National Agriculture Imagery Program (has only US data)

In [36]:
import io
from PIL import Image as pil_Image

In [176]:
# show the image on folium map
bounds = [137.3, 34.65, 137.5, 34.85]
sat_region = ee.Geometry.Rectangle(bounds)

# # 1. Sentinel
# satellite = ee.ImageCollection("COPERNICUS/S2_SR")\
#             .filterDate('2018-01-01', '2018-12-31')\
#             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))\
#             .map(maskS2clouds).select(['B4', 'B3', 'B2'])
# parameters = {'min': 0,
#               'max': 0.3,
#               #'bands': ['B4', 'B3', 'B2'],
#               'region': sat_region}

# 2. Landsat-8
satellite = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
                    .filterDate('2016-01-01', '2016-12-31')
parameters = {'min': 0,
              'max': 3000,
              # 'scale':1,
              # 'dimensions': 1024*3,
              'bands': ['B4', 'B3', 'B2'],
              'region': sat_region}

# filter area
satellite_AOI = satellite.filterBounds(sat_region)


satellite_least_cloudy = ee.Image(satellite_AOI.sort('LAND_COVER').first())


c1 = (bounds[0]+bounds[2])/2
c2 = (bounds[1]+bounds[3])/2
toyohashi_map = folium.Map(
                            location=[c2,c1],
                           zoom_start=10,
                           # min_lat=34.65,
                           # max_lat=34.85,
                           # min_lon=137.3,
                           # max_lon=137.5,
                           # max_bounds=True,
                           # zoom_control=False,
                           # png_enabled=True
                          )
# Ituna_map
toyohashi_map.addLayer(satellite_least_cloudy, parameters)

# specify map boundary
toyohashi_map.fit_bounds([[34.65, 137.3],[34.85, 137.5]])

# show map
toyohashi_map

In [177]:
toyohashi_map.save("toyohashi.html")

### Sentinel-2 MSI: MultiSpectral Instrument, Level-1C

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# setting the Area of Interest
toyohashi = ee.Geometry.Rectangle([137.3, 32.65, 137.5, 34.85])

# filter area
landsat_toyo = landsat.filterBounds(toyohashi)


# choose dates
# landsat = landsat.filterDate('2019-07-01','2019-12-01')
landsat_toyo = landsat_toyo.filterDate('2016-01-01', '2016-12-31')
# print(landsat_AOI.getInfo())

# choose image
# the least cloudy image
land_cover_toyo = ee.Image(landsat_toyo)
print(type(land_cover_toyo))

In [ ]:
# Display a thumbnail of global elevation.
Image(url = land_cover_toyo.updateMask(land_cover_toyo.gt(1))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [ ]:

# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt

# Fetch a Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

# Select Red and NIR bands, scale them, and sample 500 points.
samp_fc = img.select(['B3','B4']).divide(10000).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()